<a href="https://colab.research.google.com/github/Doradux/IABD-Course/blob/main/BDA/Ejercicios_de_metricas%20_de_clasificacion/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets download -d isabelocastillo/jugadores-de-futbol-de-la-rfef-temporada-2023-2024 -p /content
!unzip /content/sample_data/jugadores-de-futbol-de-la-rfef-temporada-2023-2024.zip -d /content

Archive:  /content/sample_data/jugadores-de-futbol-de-la-rfef-temporada-2023-2024.zip
  inflating: /content/jugadores_RFEF_2024.csv  


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max

spark = SparkSession.builder.appName("Analisis de jugadores").getOrCreate()

df = spark.read.csv("./jugadores_RFEF_2024.csv", header=True, inferSchema=True)
rdd = df.rdd

### Filtrar los jugadores que han jugado más de 20 partidos de titular y cuya edad sea menor de 25 años.

In [ ]:
ej1 = rdd.filter(lambda x: x.partidos_titular > 20 and x.edad < 25 and x.edad != 0)
ej1.take(5)

[Row(id=4204, nombre='Sergio', partidos_jugados=27, partidos_titular=27, goles=54, asistencias=0, tarjeta_amarilla=2, edad=23, altura=180.68621175325222, valor=6220.0, rating=43, posicion='Portero'),
 Row(id=1546, nombre='Buru', partidos_jugados=25, partidos_titular=23, goles=49, asistencias=0, tarjeta_amarilla=4, edad=24, altura=183.0, valor=77600.0, rating=49, posicion='Portero'),
 Row(id=4093, nombre='Javi Álvarez', partidos_jugados=27, partidos_titular=27, goles=49, asistencias=0, tarjeta_amarilla=4, edad=21, altura=191.0, valor=550.0, rating=34, posicion='Portero'),
 Row(id=7691, nombre='David García', partidos_jugados=33, partidos_titular=33, goles=49, asistencias=0, tarjeta_amarilla=3, edad=24, altura=190.0, valor=10140.0, rating=40, posicion='Portero'),
 Row(id=8388, nombre='O. Perez', partidos_jugados=34, partidos_titular=34, goles=49, asistencias=0, tarjeta_amarilla=3, edad=21, altura=180.7848123641227, valor=5740.0, rating=42, posicion='Portero')]

### Filtrar jugadores con un valor de mercado superior a una cantidad determinada (por ejemplo, 100,000).

In [ ]:
ej2= rdd.filter(lambda x: x.valor >= 100000)
ej2.take(5)

[Row(id=195, nombre='A. Ortolà', partidos_jugados=38, partidos_titular=38, goles=57, asistencias=0, tarjeta_amarilla=3, edad=30, altura=187.0, valor=230820.0, rating=54, posicion='Portero'),
 Row(id=2368, nombre='Edu Frias', partidos_jugados=32, partidos_titular=32, goles=56, asistencias=0, tarjeta_amarilla=1, edad=25, altura=184.0, valor=202510.0, rating=52, posicion='Portero'),
 Row(id=283, nombre='Ander Iru', partidos_jugados=27, partidos_titular=27, goles=44, asistencias=0, tarjeta_amarilla=3, edad=25, altura=188.0, valor=147260.0, rating=47, posicion='Portero'),
 Row(id=897, nombre='Samu Casado', partidos_jugados=31, partidos_titular=31, goles=42, asistencias=0, tarjeta_amarilla=1, edad=27, altura=187.0, valor=143540.0, rating=49, posicion='Portero'),
 Row(id=227, nombre='Taliby', partidos_jugados=38, partidos_titular=38, goles=41, asistencias=0, tarjeta_amarilla=1, edad=26, altura=190.0, valor=203650.0, rating=58, posicion='Portero')]

### Ordenar los jugadores por su valoración (campo valor).

In [ ]:
ej3 = rdd.sortBy(lambda x: x.rating, ascending=False)
ej3.take(5)

[Row(id=1067, nombre='Lucas Pérez', partidos_jugados=31, partidos_titular=31, goles=12, asistencias=17, tarjeta_amarilla=6, edad=35, altura=180.0, valor=991770.0, rating=70, posicion='Delantero Centro'),
 Row(id=444, nombre='Einar Galilea', partidos_jugados=24, partidos_titular=22, goles=2, asistencias=0, tarjeta_amarilla=5, edad=30, altura=185.0, valor=1310000.0, rating=69, posicion='Defensa Central'),
 Row(id=416, nombre='J. Martínez', partidos_jugados=8, partidos_titular=7, goles=0, asistencias=0, tarjeta_amarilla=1, edad=31, altura=191.0, valor=2140000.0, rating=68, posicion='Defensa Central'),
 Row(id=1062, nombre='Salva Sevilla', partidos_jugados=18, partidos_titular=14, goles=1, asistencias=1, tarjeta_amarilla=1, edad=40, altura=178.0, valor=115660.0, rating=67, posicion='Mediocentro'),
 Row(id=442, nombre='A. Herrero', partidos_jugados=38, partidos_titular=38, goles=25, asistencias=0, tarjeta_amarilla=3, edad=30, altura=183.0, valor=561970.0, rating=65, posicion='Portero')]

### Contar cuántos jugadores cumplen con las condiciones anteriores.

In [ ]:
ej4 ={"Ej1": ej1.count(), "Ej2": ej2.count(), "Ej3": ej3.count()}
ej4

{'Ej1': 1512, 'Ej2': 1421, 'Ej3': 10107}

### Encontrar el jugador con el valor de mercado más alto.

In [ ]:
valorDes = rdd.sortBy(lambda x: x.valor, ascending=False)
ej5 = valorDes.first()
ej5

Row(id=2771, nombre='Assane Diao', partidos_jugados=5, partidos_titular=4, goles=2, asistencias=0, tarjeta_amarilla=1, edad=18, altura=185.0, valor=3370000.0, rating=49, posicion='Extremo Izquierdo')

### Mostrar los primeros 10 jugadores ordenados por goles y asistencias.

In [ ]:
rdd = rdd.sortBy(lambda x: (x.goles, x.asistencias), ascending=False)
rdd.take(10)

[Row(id=11515, nombre='A. Lage', partidos_jugados=32, partidos_titular=32, goles=97, asistencias=0, tarjeta_amarilla=2, edad=35, altura=180.0937669559174, valor=1000.0, rating=42, posicion='Portero'),
 Row(id=8675, nombre='G. Caldentey', partidos_jugados=28, partidos_titular=27, goles=79, asistencias=0, tarjeta_amarilla=0, edad=27, altura=180.4883187803087, valor=518.52, rating=40, posicion='Portero'),
 Row(id=9238, nombre='Álvaro Iso', partidos_jugados=31, partidos_titular=30, goles=67, asistencias=0, tarjeta_amarilla=1, edad=27, altura=185.0, valor=13570.0, rating=36, posicion='Portero'),
 Row(id=11012, nombre='R. Cuba', partidos_jugados=34, partidos_titular=34, goles=63, asistencias=0, tarjeta_amarilla=1, edad=0, altura=181.82042303963172, valor=3630.0, rating=43, posicion='Portero'),
 Row(id=3740, nombre='Chema Leobalde', partidos_jugados=30, partidos_titular=30, goles=61, asistencias=0, tarjeta_amarilla=3, edad=26, altura=180.5386244442277, valor=9960.0, rating=45, posicion='Porte

### Agrupar los jugadores por posición y calcular el promedio de goles por posición.

In [ ]:
rdd2 = rdd

media_goles = (rdd2.map(lambda x: (x.posicion, x.goles)).groupByKey().mapValues(lambda goles: sum(goles) / len(goles)))

media_goles = media_goles.collect()
for posicion, media in media_goles:
    print(f"Posición: {posicion} => {media}")



Posición: Portero => 11.16564417177914
Posición: Delantero Centro => 3.9915032679738562
Posición: Extremo Derecho => 2.7236580516898607
Posición: Extremo Izquierdo => 3.228448275862069
Posición: Mediocentro => 1.4396060134784863
Posición: MI => 1.888157894736842
Posición: Defensa Central => 0.8948545861297539
Posición: MP => 2.9906103286384975
Posición: Lateral Derecho => 0.7176820208023774
Posición: Lateral Izquierdo => 0.8160377358490566
Posición: MD => 1.7279411764705883
Posición: Mediocentro Defensivo => 1.0966183574879227
Posición: CAI => 1.7142857142857142
Posición: MPI => 2.5
Posición: PT => 0.5
Posición: CAD => 0.4
Posición: MPD => 0.0


### Realizar una muestra aleatoria de jugadores de entre 20 y 30 años.

In [ ]:
rdd2 = rdd
rdd2= rdd2.filter(lambda x: x.edad >= 20 and x.edad <= 30)
rdd2 = rdd2.takeSample(False, 10)
rdd2

[Row(id=3983, nombre='M. Ndiaye', partidos_jugados=13, partidos_titular=1, goles=1, asistencias=0, tarjeta_amarilla=1, edad=21, altura=188.0, valor=1000.0, rating=25, posicion='Delantero Centro'),
 Row(id=4346, nombre='Manuel Cantero', partidos_jugados=25, partidos_titular=21, goles=0, asistencias=0, tarjeta_amarilla=5, edad=28, altura=180.4403976698514, valor=14040.0, rating=46, posicion='Defensa Central'),
 Row(id=4517, nombre='R. Ruiz', partidos_jugados=4, partidos_titular=1, goles=1, asistencias=0, tarjeta_amarilla=0, edad=22, altura=172.0, valor=3830.0, rating=40, posicion='Mediocentro'),
 Row(id=10702, nombre='Miguel Tomé', partidos_jugados=30, partidos_titular=29, goles=1, asistencias=0, tarjeta_amarilla=10, edad=23, altura=180.0, valor=9370.0, rating=40, posicion='Mediocentro Defensivo'),
 Row(id=9861, nombre='Ale Benítez', partidos_jugados=32, partidos_titular=32, goles=0, asistencias=0, tarjeta_amarilla=7, edad=24, altura=175.0, valor=22360.0, rating=45, posicion='Mediocentro